In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


## Importing Libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [3]:
train_dataset = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test_dataset = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
X_train_dev = train_dataset.loc[:,train_dataset.columns.drop(['Id','SalePrice'])].values
y_train_dev = train_dataset.iloc[:,-1].values
X_test = test_dataset.loc[:,test_dataset.columns.drop('Id')].values
output = np.zeros([X_test.shape[0],2])
output[:,0] = test_dataset.iloc[:,0]

In [4]:
print(X_train_dev)
X_train_dev_length = X_train_dev.shape[0]
print(X_train_dev_length)

[[60 'RL' 65.0 ... 2008 'WD' 'Normal']
 [20 'RL' 80.0 ... 2007 'WD' 'Normal']
 [60 'RL' 68.0 ... 2008 'WD' 'Normal']
 ...
 [70 'RL' 66.0 ... 2010 'WD' 'Normal']
 [20 'RL' 68.0 ... 2010 'WD' 'Normal']
 [20 'RL' 75.0 ... 2008 'WD' 'Normal']]
1460


In [5]:
print(y_train_dev)

[208500 181500 223500 ... 266500 142125 147500]


In [6]:
print(X_test)
X_test.shape[1]

[[20 'RH' 80.0 ... 2010 'WD' 'Normal']
 [20 'RL' 81.0 ... 2010 'WD' 'Normal']
 [60 'RL' 74.0 ... 2010 'WD' 'Normal']
 ...
 [20 'RL' 160.0 ... 2006 'WD' 'Abnorml']
 [85 'RL' 62.0 ... 2006 'WD' 'Normal']
 [60 'RL' 74.0 ... 2006 'WD' 'Normal']]


79

## Taking Care of Missing Data

In [7]:
from sklearn.impute import SimpleImputer
num_cols_to_find_train = train_dataset.select_dtypes(include='number').columns.values
num_cols_train = [train_dataset.columns.get_loc(col)-1 for col in num_cols_to_find_train]
del num_cols_train[0]
del num_cols_train[-1]
imputer_train = SimpleImputer(missing_values = np.nan, strategy = 'median')
imputer_train.fit(X_train_dev[:,num_cols_train])
X_train_dev[:,num_cols_train] = imputer_train.transform(X_train_dev[:,num_cols_train])

num_cols_to_find_test = test_dataset.select_dtypes(include='number').columns.values
num_cols_test = [test_dataset.columns.get_loc(col)-1 for col in num_cols_to_find_test]
del num_cols_test[0]
del num_cols_test[-1]
imputer_test = SimpleImputer(missing_values = np.nan, strategy = 'median')
imputer_test.fit(X_train_dev[:,num_cols_test])
X_test[:,num_cols_test] = imputer_test.transform(X_test[:,num_cols_test])

In [8]:
print(X_train_dev)
X_train_dev.shape[1]

[[60.0 'RL' 65.0 ... 2008.0 'WD' 'Normal']
 [20.0 'RL' 80.0 ... 2007.0 'WD' 'Normal']
 [60.0 'RL' 68.0 ... 2008.0 'WD' 'Normal']
 ...
 [70.0 'RL' 66.0 ... 2010.0 'WD' 'Normal']
 [20.0 'RL' 68.0 ... 2010.0 'WD' 'Normal']
 [20.0 'RL' 75.0 ... 2008.0 'WD' 'Normal']]


79

In [9]:
print(X_test)
X_test.shape[1]

[[20.0 'RH' 80.0 ... 2010 'WD' 'Normal']
 [20.0 'RL' 81.0 ... 2010 'WD' 'Normal']
 [60.0 'RL' 74.0 ... 2010 'WD' 'Normal']
 ...
 [20.0 'RL' 160.0 ... 2006 'WD' 'Abnorml']
 [85.0 'RL' 62.0 ... 2006 'WD' 'Normal']
 [60.0 'RL' 74.0 ... 2006 'WD' 'Normal']]


79

## Encoding Categorical Data

In [10]:
categorical_cols_to_find_train = train_dataset.select_dtypes(include=['object']).columns.values
categorical_cols_train = [train_dataset.columns.get_loc(col)-1 for col in categorical_cols_to_find_train]
print(categorical_cols_train)
categorical_cols_to_find_test = test_dataset.select_dtypes(include=['object']).columns.values
categorical_cols_test = [test_dataset.columns.get_loc(col)-1 for col in categorical_cols_to_find_test]
print(categorical_cols_test)

[1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 34, 38, 39, 40, 41, 52, 54, 56, 57, 59, 62, 63, 64, 71, 72, 73, 77, 78]
[1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 34, 38, 39, 40, 41, 52, 54, 56, 57, 59, 62, 63, 64, 71, 72, 73, 77, 78]


In [11]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
X = np.concatenate((X_train_dev, X_test), axis=0)
categorical_cols_to_find_train = train_dataset.select_dtypes(include=['object']).columns.values
categorical_cols_train = [train_dataset.columns.get_loc(col)-1 for col in categorical_cols_to_find_train]
ct_train = make_column_transformer((OneHotEncoder(),categorical_cols_train),remainder = 'passthrough')
X = ct_train.fit_transform(X)
X = X.todense()
print(X.shape[0])
X_train_dev, X_test = np.array_split(X, [X_train_dev_length],axis = 0)

2919


In [12]:
print(X_train_dev)
print(X_train_dev.shape[0])
print(X_train_dev.shape[1])

[[0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 2.000e+00 2.008e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 5.000e+00 2.007e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 9.000e+00 2.008e+03]
 ...
 [0.000e+00 0.000e+00 0.000e+00 ... 2.500e+03 5.000e+00 2.010e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 4.000e+00 2.010e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 6.000e+00 2.008e+03]]
1460
311


In [13]:
print(X_test)
X_test.shape[1]

[[0.000e+00 0.000e+00 1.000e+00 ... 0.000e+00 6.000e+00 2.010e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 1.250e+04 6.000e+00 2.010e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 3.000e+00 2.010e+03]
 ...
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 9.000e+00 2.006e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 7.000e+02 7.000e+00 2.006e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 1.100e+01 2.006e+03]]


311

## Splitting into dev and training sets

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, test_size = 0.1, random_state = 0) 

In [15]:
print(X_train)

[[0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 7.000e+00 2.009e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 8.000e+00 2.008e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 2.000e+00 2.010e+03]
 ...
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 4.000e+00 2.010e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 1.000e+01 2.006e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 6.000e+00 2.010e+03]]


## Feature Scaling

In [16]:
from sklearn.preprocessing import StandardScaler 
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_dev = sc.transform(X_dev)
X_test = sc.transform(X_test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn

## Training the Model

In [17]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X_train,y_train)

RandomForestRegressor(n_estimators=10, random_state=0)

## Evaluating the Model Performance

In [18]:
from sklearn.metrics import r2_score
y_dev_pred = regressor.predict(X_dev)
r2_score(y_dev,y_dev_pred)

0.8858342053346228

## Predicting Test Set Results

In [19]:
output[:,1]  = regressor.predict(X_test)
output_DF = pd.DataFrame(output.astype(int),columns=['Id','SalePrice'])
output_DF.to_csv("./ud2.csv",index=False)
check_df = pd.read_csv('./ud2.csv')
print(check_df)

        Id  SalePrice
0     1461     124830
1     1462     160425
2     1463     179028
3     1464     180265
4     1465     189905
...    ...        ...
1454  2915      85700
1455  2916      80081
1456  2917     143300
1457  2918     108275
1458  2919     244600

[1459 rows x 2 columns]
